# 10 - Graph Processing z GraphFrames

Analiza grafowa danych MovieLens za pomocą GraphFrames.

**Tematy:**
- Budowa grafu: wierzchołki i krawędzie
- Degree centrality (in-degree, out-degree)
- PageRank - ważność wierzchołków
- Connected Components
- BFS (Breadth-First Search)
- Triangle Counting
- Motif finding - wyszukiwanie wzorców w grafie
- Podobieństwo filmów przez wspólnych użytkowników

## 1. Setup

GraphFrames wymaga dodatkowej paczki Maven.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("10_Graph_Processing") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", 
            "org.postgresql:postgresql:42.7.1,"
            "graphframes:graphframes:0.8.4-spark3.5-s_2.12") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

# Jeśli graphframes nie zadziała, można zainstalować przez pip:
# !pip install graphframes

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

ratings_full = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

# Pracujemy na próbce (grafy na pełnych danych są kosztowne!)
ratings = ratings_full.filter(col("user_id") <= 5000)
ratings.cache()
print(f"Próbka ratings: {ratings.count()} (users <= 5000)")

## 2. Budowa grafu bipartytowego

Graf bipartytowy: dwa typy wierzchołków (users, movies) i krawędzie (oceny) między nimi.

```
User_1 --[4.5]--> Movie_318 (Shawshank)
User_1 --[5.0]--> Movie_296 (Pulp Fiction)
User_2 --[3.5]--> Movie_318
```

GraphFrames wymaga:
- `vertices` DataFrame z kolumną `id`
- `edges` DataFrame z kolumnami `src`, `dst`

In [ ]:
from graphframes import GraphFrame

# Wierzchołki - użytkownicy i filmy z prefiksem (aby id były unikalne)
user_vertices = ratings.select("user_id").distinct() \
    .withColumn("id", concat(lit("u_"), col("user_id"))) \
    .withColumn("type", lit("user")) \
    .withColumn("name", concat(lit("User "), col("user_id"))) \
    .select("id", "type", "name")

movie_vertices = ratings.select("movie_id").distinct() \
    .join(movies, "movie_id") \
    .withColumn("id", concat(lit("m_"), col("movie_id"))) \
    .withColumn("type", lit("movie")) \
    .withColumn("name", col("title")) \
    .select("id", "type", "name")

vertices = user_vertices.union(movie_vertices)

# Krawędzie
edges = ratings \
    .withColumn("src", concat(lit("u_"), col("user_id"))) \
    .withColumn("dst", concat(lit("m_"), col("movie_id"))) \
    .select("src", "dst", col("rating").alias("weight"))

print(f"Vertices: {vertices.count()} ({user_vertices.count()} users + {movie_vertices.count()} movies)")
print(f"Edges: {edges.count()}")

# Stwórz graf
g = GraphFrame(vertices, edges)
g.vertices.show(5)
g.edges.show(5)

## 3. Degree Centrality

- **In-degree** filmu = liczba użytkowników, którzy go ocenili (popularność)
- **Out-degree** użytkownika = liczba filmów, które ocenił (aktywność)

In [ ]:
# In-degree - ile krawędzi prowadzi DO wierzchołka
in_degrees = g.inDegrees

# Najpopularniejsze filmy (najwyższy in-degree)
popular_movies = in_degrees \
    .filter(col("id").startswith("m_")) \
    .join(g.vertices, "id") \
    .orderBy(desc("inDegree"))

print("Top 10 najpopularniejszych filmów (in-degree):")
popular_movies.select("name", "inDegree").show(10, truncate=False)

In [ ]:
# Out-degree - najaktywniejsze użytkownicy
out_degrees = g.outDegrees

active_users = out_degrees \
    .filter(col("id").startswith("u_")) \
    .join(g.vertices, "id") \
    .orderBy(desc("outDegree"))

print("Top 10 najaktywniejszych użytkowników (out-degree):")
active_users.select("name", "outDegree").show(10)

In [ ]:
# Rozkład degree
degree_dist = g.degrees
degree_dist.join(g.vertices, "id") \
    .groupBy("type") \
    .agg(
        count("*").alias("count"),
        round(avg("degree"), 1).alias("avg_degree"),
        min("degree").alias("min_degree"),
        max("degree").alias("max_degree")
    ).show()

## 4. PageRank

PageRank mierzy "ważność" wierzchołka na podstawie kto na niego wskazuje.

W kontekście MovieLens:
- Film jest ważny jeśli oceniają go aktywni użytkownicy
- Użytkownik jest ważny jeśli ocenia popularne filmy

In [ ]:
# PageRank - 10 iteracji
# resetProbability = prawdopodobieństwo "teleportacji" (damping factor = 1 - resetProbability)
pr = g.pageRank(resetProbability=0.15, maxIter=10)

# Top filmy wg PageRank
print("Top 15 filmów wg PageRank:")
pr.vertices \
    .filter(col("type") == "movie") \
    .orderBy(desc("pagerank")) \
    .select("name", round(col("pagerank"), 4).alias("pagerank")) \
    .show(15, truncate=False)

In [ ]:
# Porównaj PageRank z prostym in-degree
pr_movies = pr.vertices.filter(col("type") == "movie") \
    .select(col("id"), col("name"), round(col("pagerank"), 4).alias("pagerank"))

in_deg_movies = in_degrees.filter(col("id").startswith("m_"))

comparison = pr_movies.join(in_deg_movies, "id") \
    .withColumn("pr_rank", dense_rank().over(Window.orderBy(desc("pagerank")))) \
    .withColumn("deg_rank", dense_rank().over(Window.orderBy(desc("inDegree"))))

from pyspark.sql.window import Window

comparison = pr_movies.join(in_deg_movies, "id") \
    .orderBy(desc("pagerank"))

print("PageRank vs In-Degree (top 15):")
comparison.select("name", "pagerank", "inDegree").show(15, truncate=False)

### Zadanie 1
Uruchom PageRank z różnymi wartościami `resetProbability` (0.05, 0.15, 0.50).
Jak zmienia się ranking filmów? Który parametr daje najbardziej sensowne wyniki?

In [ ]:
# Twoje rozwiązanie:


## 5. Motif Finding - wyszukiwanie wzorców

Motif finding pozwala szukać podgrafów o określonym kształcie.

Składnia: `"(a)-[e]->(b)"` = wierzchołek `a` połączony krawędzią `e` z wierzchołkiem `b`.

In [ ]:
# Znajdź użytkowników, którzy ocenili ten sam film
# Pattern: user1 -> movie <- user2
shared_movies = g.find("(u1)-[e1]->(m); (u2)-[e2]->(m)") \
    .filter("u1.id < u2.id") \
    .filter("u1.type = 'user' AND u2.type = 'user' AND m.type = 'movie'")

# Ile par użytkowników współdzieli film?
print(f"Pary user-user ze wspólnym filmem: {shared_movies.count()}")

# Pokaż przykłady
shared_movies.select(
    col("u1.name").alias("user_1"),
    col("u2.name").alias("user_2"),
    col("m.name").alias("movie"),
    col("e1.weight").alias("rating_1"),
    col("e2.weight").alias("rating_2")
).show(10, truncate=False)

In [ ]:
# Znajdź użytkowników z identycznym gustem - ocenili 2 filmy tak samo
# Pattern: user1 -> movie1 <- user2 AND user1 -> movie2 <- user2
# (ale to jest bardzo kosztowne, pracujmy na mniejszej próbce)

# Prostsza wersja: para użytkowników, którzy WYSOKO ocenili ten sam film
high_match = g.find("(u1)-[e1]->(m); (u2)-[e2]->(m)") \
    .filter("u1.id < u2.id") \
    .filter("e1.weight >= 4.5 AND e2.weight >= 4.5") \
    .filter("u1.type = 'user' AND u2.type = 'user'")

# Które pary mają najwięcej wspólnych wysoko ocenionych filmów?
soul_mates = high_match \
    .groupBy("u1.name", "u2.name") \
    .agg(count("*").alias("shared_favorites")) \
    .orderBy(desc("shared_favorites"))

print("Pary użytkowników z najbardziej podobnym gustem (wspólne 4.5+ oceny):")
soul_mates.show(10)

## 6. Podobieństwo filmów przez wspólnych użytkowników

Dwa filmy są podobne jeśli wielu tych samych użytkowników je oceniło (i to podobnie).

In [ ]:
# Pracujemy na mniejszej próbce - top 100 najpopularniejszych filmów
top_movies_ids = ratings.groupBy("movie_id").count() \
    .orderBy(desc("count")).limit(100) \
    .select("movie_id").rdd.flatMap(lambda x: x).collect()

ratings_small = ratings.filter(col("movie_id").isin(top_movies_ids))

# Self-join: użytkownicy, którzy ocenili oba filmy
r1 = ratings_small.select(
    col("user_id"),
    col("movie_id").alias("movie_a"),
    col("rating").alias("rating_a")
)
r2 = ratings_small.select(
    col("user_id"),
    col("movie_id").alias("movie_b"),
    col("rating").alias("rating_b")
)

co_rated = r1.join(r2, "user_id") \
    .filter(col("movie_a") < col("movie_b"))

# Cosine similarity na podstawie ocen wspólnych użytkowników
movie_similarity = co_rated.groupBy("movie_a", "movie_b").agg(
    count("*").alias("common_users"),
    (sum(col("rating_a") * col("rating_b")) / 
     (sqrt(sum(col("rating_a") ** 2)) * sqrt(sum(col("rating_b") ** 2)))
    ).alias("cosine_sim")
).filter(col("common_users") >= 50)  # min 50 wspólnych użytkowników

movie_similarity.cache()
print(f"Pary filmów z similarity: {movie_similarity.count()}")

In [ ]:
# Filmy najbardziej podobne do Shawshank Redemption (318)
target_movie = 318

similar_to_target = movie_similarity \
    .filter((col("movie_a") == target_movie) | (col("movie_b") == target_movie)) \
    .withColumn("other_movie",
        when(col("movie_a") == target_movie, col("movie_b")).otherwise(col("movie_a"))
    ) \
    .join(movies.withColumnRenamed("movie_id", "other_movie"), "other_movie") \
    .orderBy(desc("cosine_sim")) \
    .select("title", "common_users", round(col("cosine_sim"), 4).alias("similarity"))

print(f"Filmy podobne do Shawshank Redemption:")
similar_to_target.show(15, truncate=False)

In [ ]:
# Zbuduj graf podobieństwa filmów (movie-movie graph)
sim_threshold = 0.9

sim_vertices = movies.filter(col("movie_id").isin(top_movies_ids)) \
    .withColumn("id", col("movie_id").cast("string")) \
    .select("id", col("title").alias("name"))

sim_edges = movie_similarity \
    .filter(col("cosine_sim") >= sim_threshold) \
    .select(
        col("movie_a").cast("string").alias("src"),
        col("movie_b").cast("string").alias("dst"),
        col("cosine_sim").alias("weight")
    )

movie_graph = GraphFrame(sim_vertices, sim_edges)
print(f"Movie similarity graph: {movie_graph.vertices.count()} vertices, {movie_graph.edges.count()} edges")
print(f"(similarity >= {sim_threshold})")

## 7. Connected Components

Znajduje grupy wierzchołków połączonych krawędziami. W movie similarity graph = klastry podobnych filmów.

In [ ]:
# Connected components wymaga checkpointa
spark.sparkContext.setCheckpointDir("/tmp/graphframes_checkpoint")

# Znajdź connected components w grafie podobieństwa filmów
cc = movie_graph.connectedComponents()

# Ile klastrów?
num_clusters = cc.select("component").distinct().count()
print(f"Liczba klastrów: {num_clusters}")

# Rozmiary klastrów
cc.groupBy("component") \
    .agg(count("*").alias("cluster_size"), collect_list("name").alias("movies")) \
    .orderBy(desc("cluster_size")) \
    .show(10, truncate=False)

## 8. BFS - Breadth-First Search

Znajdź najkrótszą ścieżkę między dwoma wierzchołkami.

In [ ]:
# BFS: znajdź ścieżkę od filmu do filmu w grafie bipartytowym
# Toy Story -> ? -> The Matrix
# (user ocenił oba filmy)

paths = g.bfs(
    fromExpr="id = 'm_1'",       # Toy Story
    toExpr="id = 'm_2571'",       # The Matrix
    maxPathLength=2               # max 2 kroki: movie -> user -> movie
)

if paths.count() > 0:
    print(f"Znaleziono {paths.count()} ścieżek Toy Story -> User -> Matrix")
    paths.select("from.name", "v1.name", "to.name").show(5, truncate=False)
else:
    print("Brak bezpośrednich ścieżek (spróbuj z większym maxPathLength)")

## 9. Triangle Counting

Liczy trójkąty przechodzące przez każdy wierzchołek. W grafie movie-movie: grupy 3 filmów wzajemnie podobnych.

In [ ]:
# Triangle counting na grafie podobieństwa filmów
# Wymaga krawędzi w obu kierunkach (undirected graph)
bidirectional_edges = sim_edges.union(
    sim_edges.select(col("dst").alias("src"), col("src").alias("dst"), "weight")
)

movie_graph_undirected = GraphFrame(sim_vertices, bidirectional_edges)

triangles = movie_graph_undirected.triangleCount()

print("Filmy z największą liczbą trójkątów (najbardziej 'centralne'):")
triangles.orderBy(desc("count")) \
    .select("name", "count") \
    .show(15, truncate=False)

### Zadanie 2
Zbuduj graf user-user, gdzie krawędź łączy dwóch użytkowników, którzy mają >= 20 wspólnych filmów.
Wagą krawędzi niech będzie cosine similarity ich ocen.

1. Znajdź connected components (grupy podobnych użytkowników)
2. Uruchom PageRank - kto jest "najbardziej wpływowym" użytkownikiem?

In [ ]:
# Twoje rozwiązanie:


## Zadanie końcowe

Zbuduj **"Movie Recommendation Graph"**:

1. Weź graf user-movie bipartytowy
2. Dla zadanego użytkownika (np. user_id=42), znajdź:
   - Filmy, które ocenił wysoko (>= 4.0)
   - Innych użytkowników, którzy też ocenili te filmy wysoko (motif finding)
   - Filmy, które ci podobni użytkownicy ocenili wysoko, ale nasz user jeszcze nie widział
3. Porównaj te rekomendacje z wynikami ALS z notebooku 08

In [ ]:
# Twoje rozwiązanie:


In [ ]:
movie_similarity.unpersist()
ratings.unpersist()
spark.stop()